In [94]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import cv2

In [ ]:
import os

folder = "UTKFace"
if not os.path.isdir(folder):
    raise FileNotFoundError(f"File not found: {folder}")

all_files = sorted([f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".png"))])
print("images count:", len(all_files))
print("first 5 images:", all_files[:5])


عدد الصور: 23708
أول 5 صور: ['100_0_0_20170112213500903.jpg.chip.jpg', '100_0_0_20170112215240346.jpg.chip.jpg', '100_1_0_20170110183726390.jpg.chip.jpg', '100_1_0_20170112213001988.jpg.chip.jpg', '100_1_0_20170112213303693.jpg.chip.jpg']


In [ ]:
import re

ages = []
bad_files = []
for file in all_files:
    m = re.match(r'^(\d+)', file) 
    if not m:
        bad_files.append(file)
        continue
    age = int(m.group(1))
    if age < 0 or age > 110:
        bad_files.append(file)
        continue
    ages.append(age)

print("toal age count:", len(ages))
print("bad age count:", len(bad_files))


إجمالي أعمار صالحة: 23700
ملفات بايظة/غير صالحة: 8


In [ ]:
import re

images = []
labels = []
for file in all_files:
    path = os.path.join(folder, file)
    img = cv2.imread(path)
    if img is None:
        print("bad image", file)
        continue

    m = re.match(r'^(\d+)', file)
    if not m:
        print("the file name is not a number", file)
        continue
    age = int(m.group(1))
    if age < 0 or age > 110:
        print("age out of range", file)
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64, 64))
    images.append(img)         
    labels.append(age)          

images = np.array(images, dtype='uint8') 
labels = np.array(labels, dtype='float32')

print("images shape:", images.shape)
print("labels shape:", labels.shape)
print("min age:", labels.min(), "max age:", labels.max())


⚠ عمر غير منطقي، تم تجاهل الملف: 111_1_0_20170120134646399.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 115_1_0_20170120134725990.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 115_1_0_20170120134725991.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 115_1_1_20170112213257263.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 116_1_0_20170112213001988.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 116_1_0_20170120134921760.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 116_1_2_20170112220255503.jpg.chip.jpg
⚠ عمر غير منطقي، تم تجاهل الملف: 116_1_3_20170120134744096.jpg.chip.jpg
شكل مصفوفة الصور: (23700, 64, 64, 3)
عدد التسميات: (23700,)
min age: 1.0 max age: 110.0


In [ ]:
import numpy as np
age_bins = np.clip(labels // 10, 0, 10)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=age_bins)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)


val_datagen = ImageDataGenerator(
    rescale=1./255  
)

train_gen = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
val_gen = val_datagen.flow(X_val, y_val, batch_size=32, shuffle=False)


In [100]:
from keras.models import Sequential   
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.optimizers import Adam     
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

In [101]:
from tensorflow.keras import regularizers

model = Sequential([
    Input(shape=(64,64,3)),
    Conv2D(32,(3,3),padding='same',activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(64,(3,3),padding='same',activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(128,(3,3),padding='same',activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.35),
    Dense(64, activation='relu'),
    Dropout(0.25),
    Dense(1, activation='linear')
])


In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='mean_absolute_error', 
              metrics=['mae'])


In [103]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("best_age_model.keras", save_best_only=True, monitor='val_loss')

In [104]:
history = model.fit(
    train_gen,
    epochs=100,
    validation_data=val_gen,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


C:\Users\ahmed\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 225s 349ms/step - loss: 18.2394 - mae: 18.2394 - val_loss: 18.2197 - val_mae: 18.2197 - learning_rate: 1.0000e-04
Epoch 2/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 206s 346ms/step - loss: 14.0397 - mae: 14.0397 - val_loss: 13.3105 - val_mae: 13.3105 - learning_rate: 1.0000e-04
Epoch 3/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 201s 339ms/step - loss: 13.3173 - mae: 13.3173 - val_loss: 12.3014 - val_mae: 12.3014 - learning_rate: 1.0000e-04
Epoch 4/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 202s 341ms/step - loss: 12.5425 - mae: 12.5425 - val_loss: 12.4991 - val_mae: 12.4991 - learning_rate: 1.0000e-04
Epoch 5/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 194s 326ms/step - loss: 11.8368 - mae: 11.8368 - val_loss: 15.4514 - val_mae: 15.4514 - learning_rate: 1.0000e-04
Epoch 6/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 195s 329ms/step - loss: 11.2904 - mae: 11.2904 - val_loss: 10.1788 - val_mae: 10.1788 - learning_rate: 1.0000e-04
Epoch 7/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 198s 334ms/step - loss: 10.93

In [105]:
model.save("age_predictor_model.keras")